导入必要的库

In [1]:
from time import sleep
from torch import nn
import random
import cv2
import torch
import os
import numpy as np
from PIL import Image
import torchvision.transforms as transforms


设置设备和随机种子

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# 设备选择，如果有可用的CUDA设备则使用CUDA，否则使用CPU

torch.manual_seed(123)
random.seed(123)
np.random.seed(10)
# 设置随机种子，使得实验结果可复现


定义图像转换

In [3]:
transform = transforms.Compose(
    [transforms.Resize((100, 100)),
     transforms.Grayscale(num_output_channels=1),  # 添加这一行
     transforms.ToTensor(),
     transforms.Normalize((0.5,), (0.5,))])


定义孪生网络模型

In [4]:
class SiameseNet(nn.Module):
    def __init__(self):
        super(SiameseNet, self).__init__()
        self.cnn = nn.Sequential(
            nn.ReflectionPad2d(1),
            # ......
            nn.Conv2d(1, 3, 3, padding=1),
            vgg19_cnn, nn.ReLU(inplace=True),
            nn.BatchNorm2d(512)
        )

        self.fc1 = nn.Sequential(
            nn.Linear(512 * 3 * 3, 1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, 1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, 512)
        )

    def forward_once(self, x):
        o = x
        o = self.cnn(o)
        o = o.reshape(x.size(0), -1)
        o = self.fc1(o)
        return o

    def forward(self, i1, i2):
        o1 = self.forward_once(i1)
        o2 = self.forward_once(i2)
        return o1, o2


加载预训练模型

In [5]:
siameseNet = torch.load('siameseNet')
siameseNet = siameseNet.to(device)
siameseNet.eval()


SiameseNet(
  (cnn): Sequential(
    (0): ReflectionPad2d((1, 1, 1, 1))
    (1): Conv2d(1, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (2): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14)

定义获取文件路径和标签的函数

In [6]:
def getFn_Dir(tpath):
    dirs = os.listdir(tpath)
    file_labels = []
    for i, dir in enumerate(dirs):
        label = i
        path2 = os.path.join(tpath, dir)
        files = os.listdir(path2)
        for file in files:
            fn = os.path.join(path2, file)
            t = (fn, label)
            file_labels.append(t)
    random.shuffle(file_labels)
    random.shuffle(file_labels)
    random.shuffle(file_labels)

    return file_labels


定义图像处理函数

In [7]:
def getImg(fn):
    img = Image.open(fn)
    img2 = img.convert('RGB')
    img = np.array(img)
    img = torch.Tensor(img)
    img = transform(img)
    return img


初始化人脸识别级联分类器

In [8]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')


计算并存储每个人脸图片的特征

In [9]:
face_features = {}

face_folder = r'.\faces'  # 你的人脸图片文件夹
for face_name in os.listdir(face_folder):
    face_img = cv2.imread(os.path.join(face_folder, face_name))
    gray = cv2.cvtColor(face_img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    if len(faces) != 1:
        continue

    (x, y, w, h) = faces[0]
    face_img = face_img[y:y+h, x:x+w]

    face_img = Image.fromarray(cv2.cvtColor(face_img, cv2.COLOR_BGR2RGB))
    face_img = transform(face_img).unsqueeze(0).to(device)
    face_feature = siameseNet.forward_once(face_img).detach().cpu().numpy()
    face_features[face_name] = face_feature

print("已录入信息")
print(face_features.keys())


已录入信息
dict_keys(['f.jpg', 'w.png'])


打开摄像头并进行实时人脸识别

In [10]:
cap = cv2.VideoCapture(0)
print('打开摄像头')

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    if len(faces) != 1:
        continue

    (x, y, w, h) = faces[0]
    face_img = frame[y:y + h, x:x + w]

    face_img = Image.fromarray(cv2.cvtColor(face_img, cv2.COLOR_BGR2RGB))
    face_img = transform(face_img).unsqueeze(0).to(device)
    face_feature = siameseNet.forward_once(face_img).detach().cpu().numpy()

    distances = {name: np.linalg.norm(feature - face_feature) for name, feature in face_features.items()}

    min_distance_name, min_distance = min(distances.items(), key=lambda x: x[1])

    threshold = 6

    if min_distance < threshold:
        cv2.putText(frame, min_distance_name, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    else:
        cv2.putText(frame, f"{min_distance_name}:{min_distance}", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2,
                    cv2.LINE_AA)

    cv2.imshow('Frame', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    sleep(0.1)

cap.release()
cv2.destroyAllWindows()


打开摄像头
